Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  out_dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  out_labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return out_dataset, out_labels

In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

In [5]:
# Define a dataset structure that will contain all sets.
datasets = {
    "image_size": 28,
    "num_labels": 10,
    "total_image_size": 28*28
}
datasets["train_data"], datasets["train_labels"] = reformat(train_dataset, train_labels)
datasets["valid_data"], datasets["valid_labels"] = reformat(valid_dataset, valid_labels)
datasets["test_data"], datasets["test_labels"] = reformat(test_dataset, test_labels)

print(datasets.keys())
print('Training set', datasets["train_data"].shape, datasets["train_labels"].shape)
print('Validation set', datasets["valid_data"].shape, datasets["valid_labels"].shape)
print('Test set', datasets["test_data"].shape, datasets["test_labels"].shape)

dict_keys(['train_data', 'total_image_size', 'valid_data', 'image_size', 'num_labels', 'train_labels', 'valid_labels', 'test_data', 'test_labels'])
Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [6]:
# Function that runs a graph.
# graph_info_ Structure that contains the computational graph
# dataset_ Dataset containing train, test and validation datasets (plus some additional data).
# batch_size_ Size of batch. 
# number_of_steps_ Number of steps to be performed.
def run_graph(graph_info_, dataset_, batch_size_, number_of_steps_):
    with tf.Session(graph=graph_info_["graph"]) as session:
        tf.global_variables_initializer().run()
        print("Initialized")
        for step in range(number_of_steps_+1):
            # Pick an offset within the training data, which has been randomized.
            # Note: we could use better randomization across epochs.
            offset = (step * batch_size_) % (dataset_["train_labels"].shape[0] - batch_size_)
            
            # Generate a minibatch.
            batch_data = dataset_["train_data"][offset:(offset + batch_size_), :]
            batch_labels = dataset_["train_labels"][offset:(offset + batch_size_), :]
            #print('Batch set', batch_data.shape, batch_labels.shape)
            
            # Prepare a dictionary telling the session where to feed the minibatch.
            # The key of the dictionary is the placeholder node of the graph to be fed,
            # and the value is the numpy array to feed to it.
            feed_dict = {graph_info_["tf_train_data"] : batch_data, graph_info_["tf_train_labels"] : batch_labels}
            # Targets to be evaluated.
            targets = [graph_info_["optimizer"], graph_info_["loss"], graph_info_["train"]]
            # Run the session and evaluate the "targets": returns loss and predictions.
            _, l, predictions = session.run(targets, feed_dict=feed_dict)
            if (step % 500 == 0):
                print("Minibatch loss at step", step, ":", l)
                print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
                print("Validation accuracy: %.1f%%" % accuracy(graph_info_["valid"].eval(), dataset_["valid_labels"]))
        print("Test accuracy: %.1f%%" % accuracy(graph_info_["test"].eval(), dataset_["test_labels"]))


Logistic regression model with SGD and L2 regularization
------

In [7]:
# Logistic regression model with SGD and L2 regularization.
def build_logistic_regression_model(datasets_, batch_size_, alpha_, l2_beta_):
    graph = tf.Graph()
    with graph.as_default():
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_data = tf.placeholder(tf.float32, shape=(batch_size_, datasets_["total_image_size"]))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size_, datasets_["num_labels"]))
        # For validation and test data - simply create constants, as they won't change.
        tf_valid_data = tf.constant(datasets_["valid_data"])
        tf_test_data = tf.constant(datasets_["test_data"])

        # Variables.
        weights = tf.Variable(tf.truncated_normal([datasets_["total_image_size"], datasets_["num_labels"]]))
        biases = tf.Variable(tf.zeros([datasets_["num_labels"]]))
  
        # Graph "leading" to logits.
        logits = tf.matmul(tf_train_data, weights) + biases
        
        # Loss fuction - built on top of the already defined logits.
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) \
            + l2_beta_*tf.nn.l2_loss(weights)
     
        info = {
            "graph": graph,
            "batch_size": batch_size_,
            "tf_train_data": tf_train_data,
            "tf_train_labels": tf_train_labels,
            "loss": loss,
            # Optimizer.
            "optimizer": tf.train.GradientDescentOptimizer(alpha_).minimize(loss),
            # Predictions for the training, validation, and test data.
            "train": tf.nn.softmax(logits),
            "valid": tf.nn.softmax(tf.matmul(tf_valid_data, weights) + biases),
            "test": tf.nn.softmax(tf.matmul(tf_test_data, weights) + biases)
        }

    return info    

In [16]:
# Build graph.
logistic_regression_graph = build_logistic_regression_model(datasets_=datasets, batch_size_=128, alpha_=0.5, l2_beta_=0.01)
# Run graph.
run_graph(logistic_regression_graph, dataset_=datasets, batch_size_=128, number_of_steps_=3000)

Initialized
Minibatch loss at step 0 : 54.1814
Minibatch accuracy: 3.1%
Validation accuracy: 8.0%
Minibatch loss at step 500 : 1.11716
Minibatch accuracy: 77.3%
Validation accuracy: 80.3%
Minibatch loss at step 1000 : 0.738345
Minibatch accuracy: 80.5%
Validation accuracy: 80.7%
Minibatch loss at step 1500 : 0.847092
Minibatch accuracy: 78.9%
Validation accuracy: 80.1%
Minibatch loss at step 2000 : 0.802329
Minibatch accuracy: 75.8%
Validation accuracy: 79.6%
Minibatch loss at step 2500 : 0.774675
Minibatch accuracy: 80.5%
Validation accuracy: 81.5%
Minibatch loss at step 3000 : 0.789026
Minibatch accuracy: 82.0%
Validation accuracy: 80.2%
Test accuracy: 87.4%


2-layer neural net with SGD and L2 regularization
------

In [17]:
# 2-layer neural net with SGD and L2 regularization.
def build_nn_2layers_model(datasets_, batch_size_, alpha_, l2_beta_):
    graph = tf.Graph()
    with graph.as_default():
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_data = tf.placeholder(tf.float32, shape=(batch_size_, datasets_["total_image_size"]))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size_, datasets_["num_labels"]))
        # For validation and test data - simply create constants, as they won't change.
        tf_valid_data = tf.constant(datasets_["valid_data"])
        tf_test_data = tf.constant(datasets_["test_data"])

        # Input -> hidden layer.
        weights1 = tf.Variable(tf.truncated_normal([datasets_["total_image_size"], 1024]))
        biases1 = tf.Variable(tf.zeros([1024])) 
        logits1 = tf.matmul(tf_train_data, weights1) + biases1
        hidden1 = tf.nn.relu(logits1)

        # hidden -> output layer.
        weights2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
        biases2 = tf.Variable(tf.zeros([num_labels])) 

        # Graph "leading" to logits.
        logits2 = tf.matmul(hidden1, weights2) + biases2  

        # Loss fuction - built on top of the already defined logits.
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2)) \
            + l2_beta_*tf.nn.l2_loss(weights1) \
            + l2_beta_*tf.nn.l2_loss(weights2)
     
        info = {
            "graph": graph,
            "batch_size": batch_size_,
            "tf_train_data": tf_train_data,
            "tf_train_labels": tf_train_labels,
            "loss": loss,
            # Optimizer.
            "optimizer": tf.train.GradientDescentOptimizer(alpha_).minimize(loss),
            # Predictions for the training, validation, and test data.
            "train": tf.nn.softmax(logits2),
            "valid": tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_data, weights1) + biases1), weights2) + biases2),
            "test": tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_data, weights1) + biases1), weights2) + biases2)
        }

    return info  
 

In [19]:
# Build graph.
nn_2layers_graph = build_nn_2layers_model(datasets_=datasets, batch_size_=128, alpha_=0.5, l2_beta_=0.01)
# Run graph.
run_graph(nn_2layers_graph, dataset_=datasets, batch_size_=128, number_of_steps_=3000)

Initialized
Minibatch loss at step 0 : 3480.83
Minibatch accuracy: 7.8%
Validation accuracy: 27.6%
Minibatch loss at step 500 : 21.4426
Minibatch accuracy: 79.7%
Validation accuracy: 83.3%
Minibatch loss at step 1000 : 0.891277
Minibatch accuracy: 84.4%
Validation accuracy: 82.9%
Minibatch loss at step 1500 : 0.772096
Minibatch accuracy: 83.6%
Validation accuracy: 83.4%
Minibatch loss at step 2000 : 0.718489
Minibatch accuracy: 81.2%
Validation accuracy: 83.3%
Minibatch loss at step 2500 : 0.753798
Minibatch accuracy: 82.8%
Validation accuracy: 83.9%
Minibatch loss at step 3000 : 0.744555
Minibatch accuracy: 83.6%
Validation accuracy: 82.9%
Test accuracy: 89.8%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [11]:
# Create "reduced" dataset.
slicedsets = datasets.copy() # Shallow copy
slice_size = 2 * 128
slicedsets["train_data"] = slicedsets["train_data"][:slice_size, :]
slicedsets["train_labels"] = slicedsets["train_labels"][:slice_size]


print(slicedsets["train_data"].shape)
print(slicedsets["train_labels"].shape)

# Build graph.
nn_2layers_graph = build_nn_2layers_model(datasets_=slicedsets, batch_size_=128, alpha_=0.5, l2_beta_=0.01)
# Run graph.
run_graph(nn_2layers_graph, dataset_=slicedsets, batch_size_=128, number_of_steps_=3000)


(256, 784)
(256, 10)
Initialized
Minibatch loss at step 0 : 3488.21
Minibatch accuracy: 14.8%
Validation accuracy: 22.9%
Minibatch loss at step 500 : 21.0058
Minibatch accuracy: 100.0%
Validation accuracy: 66.1%
Minibatch loss at step 1000 : 0.341622
Minibatch accuracy: 100.0%
Validation accuracy: 71.3%
Minibatch loss at step 1500 : 0.188954
Minibatch accuracy: 100.0%
Validation accuracy: 71.5%
Minibatch loss at step 2000 : 0.181841
Minibatch accuracy: 100.0%
Validation accuracy: 71.6%
Minibatch loss at step 2500 : 0.178564
Minibatch accuracy: 100.0%
Validation accuracy: 71.6%
Minibatch loss at step 3000 : 0.176522
Minibatch accuracy: 100.0%
Validation accuracy: 71.5%
Test accuracy: 78.7%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [12]:
# 2-layer neural net with dropout, L2 regularization, optimized with Adam.
def build_nn_2layers_dropout_model(datasets_, batch_size_, alpha_, l2_beta_, dropout_keep_prob_):
    graph = tf.Graph()
    with graph.as_default():
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_data = tf.placeholder(tf.float32, shape=(batch_size_, datasets_["total_image_size"]))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size_, datasets_["num_labels"]))
        # For validation and test data - simply create constants, as they won't change.
        tf_valid_data = tf.constant(datasets_["valid_data"])
        tf_test_data = tf.constant(datasets_["test_data"])

        # Input -> hidden layer.
        weights1 = tf.Variable(tf.truncated_normal([datasets_["total_image_size"], 1024]))
        biases1 = tf.Variable(tf.zeros([1024])) 
        logits1 = tf.matmul(tf_train_data, weights1) + biases1
        hidden1 = tf.nn.relu(logits1)
        hidden1_dropout = tf.nn.dropout(hidden1, dropout_keep_prob_)

        # hidden -> output layer.
        weights2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
        biases2 = tf.Variable(tf.zeros([num_labels])) 

        # Graph "leading" to logits.
        logits2 = tf.matmul(hidden1_dropout, weights2) + biases2  

        # Loss fuction - built on top of the already defined logits.
        loss = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits2)) \
            + l2_beta_*tf.nn.l2_loss(weights1) \
            + l2_beta_*tf.nn.l2_loss(weights2)
     
        info = {
            "graph": graph,
            "batch_size": batch_size_,
            "tf_train_data": tf_train_data,
            "tf_train_labels": tf_train_labels,
            "loss": loss,
            # Optimizer.
            "optimizer": tf.train.AdamOptimizer(alpha_).minimize(loss),
            # Predictions for the training, validation, and test data.
            "train": tf.nn.softmax(logits2),
            "valid": tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_data, weights1) + biases1), weights2) + biases2),
            "test": tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_data, weights1) + biases1), weights2) + biases2)
        }

    return info  
 

In [13]:
# Build graph.
nn_2layers_graph = build_nn_2layers_dropout_model(datasets_=datasets, batch_size_=128, 
          alpha_=0.001, l2_beta_=0.01, dropout_keep_prob_=0.5)
# Run graph.
run_graph(nn_2layers_graph, dataset_=datasets, batch_size_=128, number_of_steps_=3000)

Initialized
Minibatch loss at step 0 : 3555.08
Minibatch accuracy: 14.1%
Validation accuracy: 16.2%
Minibatch loss at step 500 : 2403.77
Minibatch accuracy: 67.2%
Validation accuracy: 81.1%
Minibatch loss at step 1000 : 1659.18
Minibatch accuracy: 70.3%
Validation accuracy: 82.6%
Minibatch loss at step 1500 : 1092.61
Minibatch accuracy: 75.8%
Validation accuracy: 82.7%
Minibatch loss at step 2000 : 698.914
Minibatch accuracy: 80.5%
Validation accuracy: 83.1%
Minibatch loss at step 2500 : 424.604
Minibatch accuracy: 78.9%
Validation accuracy: 83.0%
Minibatch loss at step 3000 : 246.152
Minibatch accuracy: 78.1%
Validation accuracy: 83.3%
Test accuracy: 89.9%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [14]:
# Creates a hidden layer.
def construct_hidden_layer(input_size_, output_size_, dropout_keep_prob_, l2_beta_, train_inputs_, valid_inputs_, test_inputs_):
    # Allocate parameteres.
    weights = tf.Variable(tf.truncated_normal([input_size_, output_size_]))
    biases = tf.Variable(tf.zeros([output_size_]))
    # Train flow: input -> output .
    train_hidden = tf.nn.relu(tf.matmul(train_inputs_, weights) + biases)
    train_hidden_dropout = tf.nn.dropout(train_hidden, dropout_keep_prob_)
    # Validation flow: input -> output.
    valid_hidden = tf.nn.relu(tf.matmul(valid_inputs_, weights) + biases)
    # Test flow: input -> output.
    test_hidden = tf.nn.relu(tf.matmul(test_inputs_, weights) + biases)
    # L2 regularization "cost".
    cost_hidden = l2_beta_ * tf.nn.l2_loss(weights);
    # Return created objects.
    return [train_hidden_dropout, valid_hidden, test_hidden, cost_hidden]


# Creates an output layer.
def construct_output_layer(input_size_, output_size_, l2_beta_, train_inputs_, valid_inputs_, test_inputs_, train_labels_):
    # Allocate parameteres.
    weights = tf.Variable(tf.truncated_normal([input_size_, output_size_]))
    biases = tf.Variable(tf.zeros([output_size_]))
    # Train flow: input -> logits .
    train_logits = tf.matmul(train_inputs_, weights) + biases  
    train_output = tf.nn.softmax(train_logits)
    # Validation flow: input -> output.
    valid_output = tf.nn.softmax(tf.matmul(valid_inputs_, weights) + biases)
    # Test flow: input -> output.
    test_output = tf.nn.softmax(tf.matmul(test_inputs_, weights) + biases)
    # Loss.
    loss_output = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=train_labels_, logits=train_logits)) #\
        #+ l2_beta_ * tf.nn.l2_loss(weights);
    # Return created objects.
    return [train_output, valid_output, test_output, loss_output]

# Builds a deep neural net with dropout, L2 regularization, optimized with Adam.
def build_dnn_model(datasets_, batch_size_, alpha_, l2_beta_, dropout_keep_prob_):
    graph = tf.Graph()
    with graph.as_default():
        # Input data. For the training data, we use a placeholder that will be fed
        # at run time with a training minibatch.
        tf_train_data = tf.placeholder(tf.float32, shape=(batch_size_, datasets_["total_image_size"]))
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size_, datasets_["num_labels"]))
        # For validation and test data - simply create constants, as they won't change.
        tf_valid_data = tf.constant(datasets_["valid_data"])
        tf_test_data = tf.constant(datasets_["test_data"])

        # Input -> hidden layer1.
        [train_hidden1, valid_hidden1, test_hidden1, loss_hidden1] = construct_hidden_layer(
            input_size_=datasets_["total_image_size"], output_size_=4096, dropout_keep_prob_=dropout_keep_prob_, 
            l2_beta_=l2_beta_, train_inputs_=tf_train_data, valid_inputs_=tf_valid_data, test_inputs_=tf_test_data)

        # Hidden layer1 -> hidden layer2.
        [train_hidden2, valid_hidden2, test_hidden2, loss_hidden2] = construct_hidden_layer(
            input_size_=4096, output_size_=2048, dropout_keep_prob_=dropout_keep_prob_, 
            l2_beta_=l2_beta_, train_inputs_=train_hidden1, valid_inputs_=valid_hidden1, test_inputs_=test_hidden1)

        # hidden layer2 -> hidden layer3.
        [train_hidden3, valid_hidden3, test_hidden3, loss_hidden3] = construct_hidden_layer(
            input_size_=2048, output_size_=1024, dropout_keep_prob_=dropout_keep_prob_, 
            l2_beta_=l2_beta_, train_inputs_=train_hidden2, valid_inputs_=valid_hidden2, test_inputs_=test_hidden2)

        # hidden layer3 -> hidden layer4.
        #[train_hidden4, valid_hidden4, test_hidden4, loss_hidden4] = construct_hidden_layer(
        #    input_size_=256, output_size_=128, dropout_keep_prob_=dropout_keep_prob_, 
        #    l2_beta_=l2_beta_, train_inputs_=train_hidden3, valid_inputs_=valid_hidden3, test_inputs_=test_hidden3)

        # Hidden layer4 -> output layer.
        [train_output, valid_output, test_output, loss_output] = construct_output_layer(
            input_size_=1024, output_size_=datasets_["num_labels"], l2_beta_=l2_beta_, 
            train_inputs_=train_hidden3, valid_inputs_=valid_hidden3, test_inputs_=test_hidden3,
            train_labels_=tf_train_labels)

        # Loss fuction - sum losses.
        #loss = loss_hidden1 + loss_hidden2 + loss_hidden3 + loss_hidden4 + loss_output
        loss = loss_output

        # Learning rate decay.
        global_step = tf.Variable(0)  # count the number of steps taken.
        decay_rate=0.99
        decay_steps=20000
        learning_rate = tf.train.exponential_decay(alpha_, global_step, decay_steps, decay_rate, staircase=True)
        optimizer = tf.train.GradientDescentOptimizer(alpha_).minimize(loss, global_step=global_step)
        
        info = {
            "graph": graph,
            "tf_train_data": tf_train_data,
            "tf_train_labels": tf_train_labels,
            "loss": loss,
            # Optimizer.
            "optimizer": optimizer, #tf.train.AdamOptimizer(alpha_).minimize(loss),
            # Predictions for the training, validation, and test data.
            "train": train_output,
            "valid": valid_output,
            "test": test_output
        }
        
    return info  
 

In [15]:
# Build graph.
dnn_graph = build_dnn_model(datasets_=datasets, batch_size_=256, \
          alpha_=0.001, l2_beta_=0.01, dropout_keep_prob_=0.5)
# Run graph.
run_graph(dnn_graph, dataset_=datasets, batch_size_=256, number_of_steps_=20000)

Initialized
Minibatch loss at step 0 : 1.01303e+06
Minibatch accuracy: 11.7%
Validation accuracy: 12.5%


KeyboardInterrupt: 